In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:

def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)




In [ ]:
def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())




In [ ]:
def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/MyDrive/intents.json') as file:
  data=json.load(file)
training_sentences=[]
training_labels=[]
labels=[]
responses=[]

In [ ]:
for intent in data['intents']:
  for pattern in intent['patterns']:
    training_sentences.append(pattern)
    training_labels.append(intent['tag'])
  responses.append(intent['responses'])
  if intent['tag'] not in labels:
    labels.append(intent['tag'])

In [ ]:
num_classes=len(labels)
lbl_encoder=LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels=lbl_encoder.transform(training_labels)

In [ ]:
vocab_size=1000
embedding_dim=16
max_len=20
oov_token="<OOV>"

In [ ]:
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sentences)
padded_sequences=pad_sequences(sequences,truncating='post',maxlen=max_len)

In [ ]:
#Model training
model=Sequential()
model.add(Embedding(vocab_size,embedding_dim,input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 37)                629       
                                                                 
Total params: 17,173
Trainable params: 17,173
Non-trainable params: 0
____________________________________________________

In [ ]:
epochs=300
history=model.fit(padded_sequences,np.array(training_labels),epochs=epochs)

Epoch 1/300
12/12 [==============================] - 1s 2ms/step - loss: 3.6118 - accuracy: 0.0336
Epoch 2/300
12/12 [==============================] - 0s 2ms/step - loss: 3.6077 - accuracy: 0.0616
Epoch 3/300
12/12 [==============================] - 0s 2ms/step - loss: 3.6046 - accuracy: 0.0616
Epoch 4/300
12/12 [==============================] - 0s 2ms/step - loss: 3.6014 - accuracy: 0.0616
Epoch 5/300
12/12 [==============================] - 0s 2ms/step - loss: 3.5984 - accuracy: 0.0616
Epoch 6/300
12/12 [==============================] - 0s 2ms/step - loss: 3.5947 - accuracy: 0.0616
Epoch 7/300
12/12 [==============================] - 0s 3ms/step - loss: 3.5902 - accuracy: 0.0616
Epoch 8/300
12/12 [==============================] - 0s 2ms/step - loss: 3.5851 - accuracy: 0.0616
Epoch 9/300
12/12 [==============================] - 0s 2ms/step - loss: 3.5789 - accuracy: 0.0616
Epoch 10/300
12/12 [==============================] - 0s 2ms/step - loss: 3.5714 - accuracy: 0.0616
Epoch 11/

In [ ]:
import pickle

In [ ]:
# to save the trained model
model.save("chat_model")
import pickle

In [ ]:
# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
 pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
  pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
pip install colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

In [ ]:
import colorama
colorama.init()
from colorama import Fore,Style,Back

In [ ]:
import random 
import pickle

In [ ]:
with open('/content/drive/MyDrive/intents.json') as file:
  data=json.load(file)

In [ ]:
def chat():
  #load trained model
  model=keras.models.load_model('chat_model')
  #load tokenizer object
  with open('tokenizer.pickle','rb') as handle:
     tokenizer=pickle.load(handle)
  #load label encoder object
  with open('label_encoder.pickle','rb') as enc:
     lbl_encoder=pickle.load(enc)
  #parameters
  max_len=20
  while True:
      print(Fore.LIGHTBLUE_EX+'User:'+Style.RESET_ALL,end="")
      inp=input()
      if inp.lower()=='q':
        break
      result=model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),truncating='post',maxlen=max_len))
      tag=lbl_encoder.inverse_transform([np.argmax(result)])
      for i in data['intents']:
         if i['tag']==tag:
           print(Fore.GREEN+'LoBot:'+Style.RESET_ALL,np.random.choice(i['responses']))
print(Fore.YELLOW+'Start messaging with the bot(type quit to stop)!'+Style.RESET_ALL)
chat()

Start messaging with the bot(type quit to stop)!
User:hi
1/1 [==============================] - 0s 94ms/step
LoBot: Hello!
User:hoe are you
1/1 [==============================] - 0s 30ms/step
LoBot: I'm LOBOT
User:q
